# MNIST data 유사도 체크를 통한 특이한 예측법

In [1]:
import tensorflow as tf

In [60]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [61]:
index_list = []
train_list = []
error_list = []
for i in list(set(train_labels)):
    index_array = np.where(train_labels==i)[0]
    index_list.append(index_array)
    # train set에서 같은 정답을 추출해서 28x28행렬을 다 더해서 평균냄.
    train_array = train_images[index_array].sum(axis=0)/len(train_images[index_array])
    train_list.append(train_array)
    # 각 샘플별로 train_array와 성분별로 얼마나 차이나는지 제곱으로 해서 전부 sum함. 그리고 샘플 평균 구하기(error구하는 느낌.) sum으로 할까 mean으로 할까?)
    error_list.append(((train_array - train_images[index_array])**2).sum(axis=1).sum(axis=1).mean())
#train_list들과 새로운 샘플과의 오차를 구해보고 가장 작은 오차를 보이는게 정답?

## squared error를 이용하여 성분의 유사도 체크 후 전체 sum이 작은 값으로 정답 설정

In [62]:
acc_list = []
for index in range(6000):
    pred = ((train_list - train_images[index])**2).sum(axis=1).sum(axis=1).argmin()
    sol = train_labels[index]
    acc_list.append(True if pred == sol else False)
False_index = np.where(np.array(acc_list) == False)[0]
1-len(False_index)/len(acc_list)    

0.813

- train set 정확도 : 0.813

In [63]:
acc_list = []
for index in range(6000):
    pred = ((train_list - test_images[index])**2).sum(axis=1).sum(axis=1).argmin()
    sol = test_labels[index]
    acc_list.append(True if pred == sol else False)
False_index = np.where(np.array(acc_list) == False)[0]
1-len(False_index)/len(acc_list)    

0.7893333333333333

- test set 정확도 : 0.789

- 오답은 왜 나온거지.. 글쎄

In [6]:
index =11
pred_ = np.array([((train_list[j] - train_images[index])**2).sum() for j in range(10)])
pred = pred_.argmin()
sol = train_labels[index]
(pred, sol)

(1, 5)

In [7]:
pred_

array([5115819.32098976, 2635133.08608766, 3269188.67510432,
       4222495.29703986, 3540480.62398504, 3429094.80486756,
       3946658.98342645, 3892022.69752059, 3625443.59572105,
       4037624.74940793])

## 위치의 유무는 어떨까?
- 잘 안돼....

In [187]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [248]:
sigmoid(train_list[1]).mean()

0.6832692891038881

In [296]:
train_list_make_ones = []
for k in range(len(train_list)):
    train_list_sigmoid = sigmoid(train_list[k] - 0.5)
    make_zero = np.where(train_list_sigmoid <0.5,0,train_list_sigmoid)
    make_zero_one = np.where(make_zero >0.5,1,make_zero)
    train_list_make_ones.append(make_zero_one)

In [295]:
def make_ones(test_image):
    train_list_sigmoid = sigmoid(test_image - 0.5)
    make_zero = np.where(train_list_sigmoid <0.5,0,train_list_sigmoid)
    make_zero_one = np.where(make_zero >0.5,1,make_zero)
    return make_zero_one

In [297]:
make_ones(train_images[10])[4]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [298]:
train_list_make_ones[j][4]

array([0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.])

In [323]:
i=4
np.array([np.where(train_list_make_ones[j] == make_ones(train_images[i]))[0].shape for j in range(10)]).argmax()

1

In [325]:
train_labels[4]

9

In [330]:
28*28

784

## data normalization 후 ..

In [64]:
#train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5 # Normalize the images to [-1, 1]

In [66]:
index_list = []
train_list = []
error_list = []
for i in list(set(train_labels)):
    index_array = np.where(train_labels==i)[0]
    index_list.append(index_array)
    # train set에서 같은 정답을 추출해서 28x28행렬을 다 더해서 평균냄.
    train_array = train_images[index_array].sum(axis=0)/len(train_images[index_array])
    train_list.append(train_array)
    # 각 샘플별로 train_array와 성분별로 얼마나 차이나는지 제곱으로 해서 전부 sum함. 그리고 샘플 평균 구하기(error구하는 느낌.) sum으로 할까 mean으로 할까?)
    error_list.append(((train_array - train_images[index_array])**2).sum(axis=1).sum(axis=1).mean())
    
    #train_list들과 새로운 샘플과의 오차를 구해보고 가장 작은 오차를 보이는게 정답?

In [68]:
acc_list = []
for index in range(6000):
    pred = ((train_list - train_images[index])**2).sum(axis=1).sum(axis=1).argmin()
    sol = train_labels[index]
    acc_list.append(True if pred == sol else False)
False_index = np.where(np.array(acc_list) == False)[0]
1-len(False_index)/len(acc_list)   

0.813

In [69]:
acc_list = []
for index in range(6000):
    pred = ((train_list - test_images[index])**2).sum(axis=1).sum(axis=1).argmin()
    sol = test_labels[index]
    acc_list.append(True if pred == sol else False)
False_index = np.where(np.array(acc_list) == False)[0]
1- len(False_index)/len(acc_list)    

0.6268333333333334

## 데이터 전처리 안하고 한 첫번째 방법이 제일 정확도가 높다. NN의 사진 학습이 높은 정확도를 가진 신기한 알고리즘인것 같다.